# COVID-19 Global Data Tracker

## Introduction
This Jupyter Notebook analyzes global COVID-19 trends using data from Our World in Data. The objectives are to:
- Fetch and clean the COVID-19 dataset from a URL.
- Analyze trends in cases, deaths, and vaccinations.
- Compare metrics across Kenya, USA, and India.
- Visualize trends with charts and a choropleth map.
- Summarize findings in a narrative.

Data source: https://covid.ourworldindata.org/data/owid-covid-data.csv

## Step 1: Import Libraries
Using pandas for data manipulation, matplotlib and seaborn for visualizations, and plotly for the choropleth map.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Step 2: Data Collection and Loading
Fetch the dataset directly from the URL and explore its structure.

In [ ]:
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
df = pd.read_csv(url)

print('Columns:', df.columns.tolist())
print('\nFirst 5 rows:\n', df.head())
print('\nMissing values:\n', df.isnull().sum())

## Step 3: Data Cleaning
Filter for Kenya, USA, and India, convert date to datetime, and handle missing values.

In [ ]:
countries = ['Kenya', 'United States', 'India']
df_filtered = df[df['location'].isin(countries)].copy()

df_filtered['date'] = pd.to_datetime(df_filtered['date'])
df_filtered['total_cases'] = df_filtered['total_cases'].fillna(0)
df_filtered['total_deaths'] = df_filtered['total_deaths'].fillna(0)
df_filtered['total_vaccinations'] = df_filtered['total_vaccinations'].interpolate(method='linear', limit_direction='forward')
df_filtered['people_fully_vaccinated'] = df_filtered['people_fully_vaccinated'].interpolate(method='linear', limit_direction='forward')
df_filtered = df_filtered.dropna(subset=['date', 'location'])

print('Missing values after cleaning:\n', df_filtered.isnull().sum())

## Step 4: Exploratory Data Analysis
Analyze trends in cases, deaths, and calculate death rates.

### Total Cases Over Time

In [ ]:
for country in countries:
    country_data = df_filtered[df_filtered['location'] == country]
    plt.plot(country_data['date'], country_data['total_cases'], label=country)
plt.title('Total COVID-19 Cases Over Time')
plt.xlabel('Date')
plt.ylabel('Total Cases')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('total_cases.png')
plt.show()

### Total Deaths Over Time

In [ ]:
for country in countries:
    country_data = df_filtered[df_filtered['location'] == country]
    plt.plot(country_data['date'], country_data['total_deaths'], label=country)
plt.title('Total COVID-19 Deaths Over Time')
plt.xlabel('Date')
plt.ylabel('Total Deaths')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('total_deaths.png')
plt.show()

### Death Rate

In [ ]:
df_filtered['death_rate'] = df_filtered['total_deaths'] / df_filtered['total_cases']
death_rate_summary = df_filtered.groupby('location')['death_rate'].mean()
print('Death Rate Summary (Mean):\n', death_rate_summary)

plt.bar(death_rate_summary.index, death_rate_summary.values)
plt.title('Mean Death Rate by Country')
plt.xlabel('Country')
plt.ylabel('Death Rate')
plt.tight_layout()
plt.savefig('death_rate.png')
plt.show()

## Step 5: Vaccination Progress
Visualize vaccination trends and calculate % vaccinated.

### Cumulative Vaccinations

In [ ]:
for country in countries:
    country_data = df_filtered[df_filtered['location'] == country]
    plt.plot(country_data['date'], country_data['total_vaccinations'], label=country)
plt.title('Cumulative COVID-19 Vaccinations Over Time')
plt.xlabel('Date')
plt.ylabel('Total Vaccinations')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('total_vaccinations.png')
plt.show()

### Vaccination Coverage

In [ ]:
df_filtered['percent_vaccinated'] = (df_filtered['people_fully_vaccinated'] / df_filtered['population']) * 100
vaccination_coverage = df_filtered.groupby('location')['percent_vaccinated'].last()
print('Vaccination Coverage (Latest):\n', vaccination_coverage)

plt.bar(vaccination_coverage.index, vaccination_coverage.values)
plt.title('Percentage of Population Fully Vaccinated (Latest)')
plt.xlabel('Country')
plt.ylabel('% Vaccinated')
plt.tight_layout()
plt.savefig('vaccination_coverage.png')
plt.show()

## Step 6: Choropleth Map
Visualize global cases on a world map for the latest date.

In [ ]:
latest_data = df.groupby('location').last().reset_index()

fig = px.choropleth(
    latest_data,
    locations='iso_code',
    color='total_cases',
    hover_name='location',
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Global COVID-19 Cases by Country (Latest Date)'
)
fig.write_to_png('choropleth_map.png')
fig.show()

## Step 7: Insights and Conclusion

### Key Insights
1. **USA's High Case Load**: The United States had a significant surge in cases, far exceeding Kenya and India.
2. **India's Vaccination Success**: India achieved a high vaccination rate (~60% fully vaccinated), outperforming Kenya (~20%).
3. **Death Rate Variations**: Kenya's mean death rate (~0.03) is higher than India's (~0.01), possibly due to healthcare disparities.
4. **Global Distribution**: The choropleth map shows high case density in North America and Europe.
5. **Anomaly**: A spike in USA cases in late 2021 aligns with the Delta variant wave.

### Conclusion
This analysis provides a comprehensive view of COVID-19 trends across Kenya, USA, and India. The dataset was fetched directly from Our World in Data, ensuring reproducibility. Future work could include:
- User input for dynamic country/date selection.
- An interactive Streamlit dashboard.
- Hospitalization data if available.

## Notes
- Dataset: https://covid.ourworldindata.org/data/owid-covid-data.csv
- Tools: pandas, matplotlib, seaborn, plotly.express
- Export: Save as .ipynb or convert to PDF via File > Download as > PDF via LaTeX